In [39]:
import pandas as pd
import numpy as np
import scipy.stats
from statistics import NormalDist
import statsmodels.api as sm

In [47]:
data = pd.read_excel('dataset.xlsx')

In [8]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [9]:
print(mean_confidence_interval(data['Salary']))

(78509.70297029703, 76435.37114430607, 80584.03479628799)


In [12]:

def confidence_interval(data, confidence=0.95):
  dist = NormalDist.from_samples(data)
  z = NormalDist().inv_cdf((1 + confidence) / 2.)
  h = dist.stdev * z / ((len(data) - 1) ** .5)
  return dist.mean - h, dist.mean + h

In [13]:
print(confidence_interval(data['Salary']))

(76436.83296863642, 80582.57297195765)


In [18]:
women = data.query('Gender == "Female"')['Salary']
men = data.query('Gender == "Male"')['Salary']

t, p = scipy.stats.ttest_ind(women, men, equal_var=False)
t


0.8369283766204882

In [25]:
cus1 = data.query('Customer == 1')['Salary']
cus2 = data.query('Customer == 0')['Salary']
(cus1.mean(), cus2.mean())


(80556.05095541402, 75145.5497382199)

In [24]:
t, p = scipy.stats.ttest_ind(cus1, cus2, equal_var=False)
(t, p)

(2.5783370535400754, 0.01008661178472906)

In [28]:
s1 = data.query('Department == "IT"')['Salary']
s2 = data.query('Department == "Sales"')['Salary']
t, p = scipy.stats.ttest_ind(s1, s2, equal_var=False)
p


0.06327390627346907

In [32]:
salary = data['Salary']
tenure = data['Tenure']

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(tenure, salary)
r_value * r_value

0.21506196126246208

In [33]:
(slope, intercept)

(2756.993673491282, 55408.27974380522)

In [44]:
data[['no_custom', 'custom']] = pd.get_dummies(data['Customer'])
data = data.drop('no_custom', axis = 1)
data['female'] = pd.get_dummies(data['Gender'])['Female']
data.head()

,Emp_ID,Department,Title,Salary,Gender,Age,Tenure,Customer,Staff,custom,intercept,female
0,4362,Sales,Senior Manager,81000,Female,43,9,1,19,1,1,1
1,6366,Sales,Manager,61000,Male,25,2,1,7,1,1,0
2,6551,Sales,Director,131000,Male,63,12,1,19,1,1,0
3,6678,Sales,Manager,71000,Female,37,11,1,6,1,1,1
4,7040,Sales,Director,125000,Female,64,9,1,22,1,1,1


In [45]:
data['intercept'] = 1

logit_mod = sm.Logit(data['custom'], data[['intercept', 'female']])
result = logit_mod.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.295400
         Iterations 4


/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/homebrew/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3511: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.llf/self.llnull


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 custom   No. Observations:                 1010
Model:                          Logit   Df Residuals:                     1008
Method:                           MLE   Df Model:                            1
Date:                Sun, 05 Feb 2023   Pseudo R-squ.:                     inf
Time:                        16:01:46   Log-Likelihood:                -298.35
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.5291      0.090      5.861      0.000       0.352       0.706
female        -0.0665      0.130     -0.512      0.609      -0.321       0.188
==============================================================================
"""

In [67]:
data2 = data
data2[['Manager', 'Dicector', 'Executive', 'Senior Manager','Technician']] = pd.get_dummies(data2['Title'])
data2[['no_custom', 'custom']] = pd.get_dummies(data2['Customer'])
data2 = data2.drop('no_custom', axis = 1)


In [66]:
data2.sample(15)

,Emp_ID,Department,Title,Salary,Gender,Age,Tenure,Customer,Staff,Manager,Dicector,Executive,Senior Manager,Technician,no_custom,custom
10,7363,IT,Technician,35600,Male,39,7,1,1,0,0,0,0,1,0,1
662,5890,IT,Manager,78600,Female,40,9,1,11,0,0,1,0,0,0,1
804,5460,Sales,Senior Manager,92500,Female,32,6,1,14,0,0,0,1,0,0,1
9,3617,IT,Technician,35600,Male,39,11,0,5,0,0,0,0,1,1,0
883,4558,Sales,Director,126300,Male,46,8,1,13,1,0,0,0,0,0,1
465,5569,IT,Manager,67700,Male,23,2,0,6,0,0,1,0,0,1,0
401,6260,Finance,Manager,64700,Male,31,3,1,11,0,0,1,0,0,0,1
957,5507,IT,Director,152000,Female,38,5,0,11,1,0,0,0,0,1,0
538,3580,Sales,Senior Manager,71600,Male,37,13,1,12,0,0,0,1,0,0,1
566,1780,Sales,Senior Manager,73000,Female,33,11,1,10,0,0,0,1,0,0,1


In [71]:
data2['intercept'] = 1

logit_mod = sm.Logit(data2['custom'], data2[['intercept', 'Manager', 'Dicector', 'Executive', 'Senior Manager','Technician']])
result = logit_mod.fit()
result.summary2()

Optimization terminated successfully.
         Current function value: 0.293650
         Iterations 9


/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/homebrew/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/homebrew/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3511: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.llf/self.llnull


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Logit
=============================================================================
Model:                    Logit                Pseudo R-squared:     inf     
Dependent Variable:       custom               AIC:                  605.1732
Date:                     2023-02-05 16:31     BIC:                  634.6794
No. Observations:         1010                 Log-Likelihood:       -296.59 
Df Model:                 5                    LL-Null:              0.0000  
Df Residuals:             1004                 LLR p-value:          1.0000  
Converged:                1.0000               Scale:                1.0000  
No. Iterations:           9.0000                                             
-----------------------------------------------------------------------------
                Coef.    Std.Err.      z    P>|z|      [0.025       0.975]   
-----------------------------------------------------------------------------
intercept       0.4651 3313798.9200  0.0000 1.0000 -6494926.0701 6494927.0003
Manager         0.5282 3313798.9200  0.0000 1.0000 -6494926.0070 6494927.0634
Dicector        0.0458 3313798.9200  0.0000 1.0000 -6494926.4894 6494926.5810
Executive      -0.1404 3313798.9200 -0.0000 1.0000 -6494926.6756 6494926.3948
Senior Manager  0.0383 3313798.9200  0.0000 1.0000 -6494926.4969 6494926.5735
Technician     -0.0068 3313798.9200 -0.0000 1.0000 -6494926.5420 6494926.5285
=============================================================================

"""

In [69]:
np.exp(result.params)

intercept         1.592111
Manager           1.695861
Dicector          1.046828
Executive         0.869011
Senior Manager    1.039001
Technician        0.993269
dtype: float64

In [63]:
1/_

intercept         0.628097
Manager           0.589671
Dicector          0.955267
Executive         1.150734
Senior Manager    0.962463
Technician        1.006776
dtype: float64

In [70]:
data2.head()

,Emp_ID,Department,Title,Salary,Gender,Age,Tenure,Customer,Staff,Manager,Dicector,Executive,Senior Manager,Technician,custom,intercept
0,4362,Sales,Senior Manager,81000,Female,43,9,1,19,0,0,0,1,0,1,1
1,6366,Sales,Manager,61000,Male,25,2,1,7,0,0,1,0,0,1,1
2,6551,Sales,Director,131000,Male,63,12,1,19,1,0,0,0,0,1,1
3,6678,Sales,Manager,71000,Female,37,11,1,6,0,0,1,0,0,1,1
4,7040,Sales,Director,125000,Female,64,9,1,22,1,0,0,0,0,1,1


In [74]:
data3 = data
print(data3.groupby('Title')['Customer'].sum())

Title
Director          108
Executive           5
Manager           202
Senior Manager    177
Technician        136
Name: Customer, dtype: int64


In [75]:
print(data3.groupby('Title').sum())

                 Emp_ID    Salary    Age  Tenure  Customer  Staff  Manager  \
Title                                                                        
Director         792770  21216900   7540    1923       108   2546      148   
Executive         34481   1652800    479     162         5    386        0   
Manager         1867430  22698600  10829    1940       202   2521        0   
Senior Manager  1589818  23167000  12219    3099       177   3574        0   
Technician      1175004  10559500   7014    1339       136    587        0   

                Dicector  Executive  Senior Manager  Technician  no_custom  \
Title                                                                        
Director               0          0               0           0         40   
Executive              8          0               0           0          3   
Manager                0        348               0           0        146   
Senior Manager         0          0             284           0

/var/folders/jj/wwz8qyq55hd4g3_rdkggc7900000gn/T/ipykernel_57686/4220823498.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(data3.groupby('Title').sum())


In [77]:
from sklearn.linear_model import LinearRegression

In [78]:
data4 = data
model = LinearRegression()

#define predictor and response variables
X, y = data4[["Age", "Staff", "Customer"]], data4[["Salary"]]

#fit regression model
model.fit(X, y)

#calculate R-squared of regression model
r_squared = model.score(X, y)

#view R-squared value
print(r_squared)

0.583589406705175
